In [73]:
#install requirements
%pip install -r ../requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [ ]:
#import data
import pandas as pd



df = pd.read_csv("../data/raw/fourth_down_2023.csv")

df =  df[df['down'] == 4]


/var/folders/_2/q8fdvpk1795d3b3yskjcv6gm0000gn/T/ipykernel_13319/3416212954.py:4: DtypeWarning: Columns (49,179,180,189,190,213,214,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/fourth_down_2023.csv")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Filter just the 'go for it' plays
go_df = df[df['play_type'] == 'pass']

X_go = go_df[["ydstogo", "score_differential", "game_seconds_remaining", "qtr", "half_seconds_remaining", "posteam_timeouts_remaining", "yardline_100"]]
Y_go_ep = go_df["ep"]  # Expected points

go_model_ep = RandomForestRegressor()
go_model_ep.fit(X_go, Y_go_ep)

Y_go_epa = go_df["epa"] # Expected points added from play

go_model_epa = RandomForestRegressor()
go_model_epa.fit(X_go, Y_go_epa)

Y_go_wp = go_df["wp"] # win probability

go_model_wp = RandomForestRegressor()
go_model_wp.fit(X_go, Y_go_wp)



RandomForestRegressor()

In [48]:
from sklearn.ensemble import RandomForestRegressor

# Filter just the 'punt' plays
punt_df = df[df['play_type'] == 'punt']

X_punt = punt_df[["ydstogo", "score_differential", "game_seconds_remaining", "qtr", "half_seconds_remaining", "posteam_timeouts_remaining", "yardline_100"]]
y_punt = punt_df["ep"]  # or WinProbAfter

punt_model_ep = RandomForestRegressor()
punt_model_ep.fit(X_punt, y_punt)

y_punt = punt_df["epa"]  # or WinProbAfter

punt_model_epa = RandomForestRegressor()
punt_model_epa.fit(X_punt, y_punt)

y_punt = punt_df["wpa"]  # or WinProbAfter

punt_model_wp = RandomForestRegressor()
punt_model_wp.fit(X_punt, y_punt)

RandomForestRegressor()

In [47]:
from sklearn.ensemble import RandomForestRegressor

# Filter just the 'go for it' plays
field_goal_df = df[df['play_type'] == 'field_goal']

X_field_goal = field_goal_df[["ydstogo", "score_differential", "game_seconds_remaining", "qtr", "half_seconds_remaining", "posteam_timeouts_remaining", "yardline_100"]]
y_field_goal = field_goal_df["ep"]  # or WinProbAfter

field_goal_model_ep = RandomForestRegressor()
field_goal_model_ep.fit(X_field_goal, y_field_goal)

y_field_goal = field_goal_df["epa"]

field_goal_model_epa = RandomForestRegressor()
field_goal_model_epa.fit(X_field_goal, y_field_goal)

y_field_goal = field_goal_df["wp"]

field_goal_model_wp = RandomForestRegressor()
field_goal_model_wp.fit(X_field_goal, y_field_goal)

RandomForestRegressor()

In [72]:
#testing https://www.nfl.com/games/packers-at-eagles-2024-reg-1?active-tab=stats


pbp = pd.DataFrame({"ydstogo":[11,11], "score_differential":[0,0], "game_seconds_remaining":[3407,3094], "qtr":[1,1], "half_seconds_remaining":[1607,1294], "posteam_timeouts_remaining":[3,3], "yardline_100":[43,13]})
play = ['Eagles vs Green bay, 4th & 11 at PHI 43. Outcome: Punt', 'Eagles vs Green bay, 4th & 11 at PHI 13. Outcome: Field Goal']
#print(pbp.loc[0])
for x in range(len(pbp)):
    print(play[x])
    go_ep, go_epa, go_wp = go_model_ep.predict(pbp.loc[[x]]), go_model_epa.predict(pbp.loc[[x]]), go_model_wp.predict(pbp.loc[[x]])
    punt_ep, punt_epa, punt_wp = punt_model_ep.predict(pbp.loc[[x]]), punt_model_epa.predict(pbp.loc[[x]]), punt_model_wp.predict(pbp.loc[[x]])
    fg_ep, fg_epa, fg_wp = field_goal_model_ep.predict(pbp.loc[[x]]), field_goal_model_epa.predict(pbp.loc[[x]]), field_goal_model_wp.predict(pbp.loc[[x]])
    print(f'Pass Expected Points: {go_ep}, Expected Points Added: {go_epa}, Win Probability: {go_wp * 100}%')
    print(f'Punt Expected Points: {punt_ep}, Expected points Added: {punt_epa}, Win Probability: {punt_wp* 100}%')
    print(f'Field goal Expected Points: {fg_ep}, Expected Points Added: {fg_epa}, Win Probability: {fg_wp * 100}%\n')
    options = {"Punt":punt_wp, "Field Goal":fg_wp, "Pass":go_wp}
    best_play = max(options, key=options.get)
    print(f'Recommended Play: {best_play}')


Eagles vs Green bay, 4th & 11 at PHI 43. Outcome: Punt
Pass Expected Points: [-0.07177294], Expected Points Added: [0.71933276], Win Probability: [49.83947742]%
Punt Expected Points: [-0.07397115], Expected points Added: [-0.35887133], Win Probability: [0.55170608]%
Field goal Expected Points: [0.08550012], Expected Points Added: [-2.01040299], Win Probability: [52.29519051]%

Recommended Play: Field Goal
Eagles vs Green bay, 4th & 11 at PHI 13. Outcome: Field Goal
Pass Expected Points: [2.88878826], Expected Points Added: [0.88449984], Win Probability: [61.72663304]%
Punt Expected Points: [0.28264355], Expected points Added: [-0.01639993], Win Probability: [-2.83501291]%
Field goal Expected Points: [2.80427633], Expected Points Added: [0.18899518], Win Probability: [54.4568792]%

Recommended Play: Pass


In [74]:
#Save model
from joblib import dump

models = {
    "field_goal_ep": field_goal_model_ep,
    "field_goal_epa": field_goal_model_epa,
    "field_goal_wp": field_goal_model_wp,
    "punt_ep": punt_model_ep,
    "punt_epa": punt_model_epa,
    "punt_wp": punt_model_wp,
    "go_ep": go_model_ep,
    "go_epa": go_model_epa,
    "go_wp": go_model_wp,
}

dump(models, "models.pkl")

['models.pkl']